## Deliverable 2. Create a Customer Travel Destinations Map.

In [39]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [40]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_Database.csv")
city_data_df.head()

,WeatherPy_Database.csv,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Rikitea,-23.1203,-134.9692,77.43,76,16,13.15,PF,2021-02-07 07:20:45
1,1,Atuona,-9.8000,-139.0333,79.63,78,81,23.42,PF,2021-02-07 07:20:45
2,2,Mar Del Plata,-38.0023,-57.5575,64.99,88,75,8.10,AR,2021-02-07 07:20:46
3,3,Butaritari,3.0707,172.7902,81.50,79,100,19.44,KI,2021-02-07 07:20:46
4,4,Sorland,67.6670,12.6934,35.76,79,76,15.59,NO,2021-02-07 07:20:46


In [41]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 

min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 65
What is the maximum temperature you would like for your trip? 75


In [42]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)


,WeatherPy_Database.csv,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
16,16,Port Alfred,-33.5906,26.8910,73.99,79,29,5.99,ZA,2021-02-07 07:20:47
26,26,Cape Town,-33.9258,18.4232,73.00,60,75,21.85,ZA,2021-02-07 07:16:35
34,34,Hilo,19.7297,-155.0900,69.80,73,90,4.61,US,2021-02-07 07:20:50
35,35,Luderitz,-26.6481,15.1594,74.32,47,0,24.85,NaN,2021-02-07 07:20:50
37,37,Airai,-8.9266,125.4092,68.79,98,100,3.27,TL,2021-02-07 07:20:50
42,42,Bredasdorp,-34.5322,20.0403,68.00,73,100,5.75,ZA,2021-02-07 07:20:51
46,46,Igurubi,-4.0000,33.7000,70.75,82,100,7.81,TZ,2021-02-07 07:20:52
52,52,Hisua,24.8333,85.4167,71.60,46,0,6.91,IN,2021-02-07 07:20:53
66,66,Kaeo,-35.1000,173.7833,66.99,69,28,5.01,NZ,2021-02-07 07:20:55
67,67,Port Elizabeth,-33.9180,25.5701,73.40,68,40,17.27,ZA,2021-02-07 07:20:55


In [43]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.notnull().sum()

WeatherPy_Database.csv    92
City                      92
Lat                       92
Lng                       92
Max Temp                  92
Humidity                  92
Cloudiness                92
Wind Speed                92
Country                   91
Date                      92
dtype: int64

In [44]:
preferred_cities_df.count()

WeatherPy_Database.csv    92
City                      92
Lat                       92
Lng                       92
Max Temp                  92
Humidity                  92
Cloudiness                92
Wind Speed                92
Country                   91
Date                      92
dtype: int64

In [45]:
preferred_cities_df.isnull().sum()

WeatherPy_Database.csv    0
City                      0
Lat                       0
Lng                       0
Max Temp                  0
Humidity                  0
Cloudiness                0
Wind Speed                0
Country                   1
Date                      0
dtype: int64

In [46]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_data = preferred_cities_df.dropna() 

In [47]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_data [["City", "Country", "Max Temp", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
16,Port Alfred,ZA,73.99,-33.5906,26.8910,
26,Cape Town,ZA,73.00,-33.9258,18.4232,
34,Hilo,US,69.80,19.7297,-155.0900,
37,Airai,TL,68.79,-8.9266,125.4092,
42,Bredasdorp,ZA,68.00,-34.5322,20.0403,
46,Igurubi,TZ,70.75,-4.0000,33.7000,
52,Hisua,IN,71.60,24.8333,85.4167,
66,Kaeo,NZ,66.99,-35.1000,173.7833,
67,Port Elizabeth,ZA,73.40,-33.9180,25.5701,
68,Arraial Do Cabo,BR,71.80,-22.9661,-42.0278,


In [48]:
len(hotel_df)

91

In [49]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.interrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Set the parameters to search for a hotel in Paris.
params["location"] = f"{lat}, {lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    
try:
    hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["names"]
except (IndexError):
    print("Hotel not found... skipping")
 

AttributeError: 'DataFrame' object has no attribute 'interrows'

In [50]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.head()

,City,Country,Max Temp,Lat,Lng,Hotel Name
16,Port Alfred,ZA,73.99,-33.5906,26.8910,
26,Cape Town,ZA,73.00,-33.9258,18.4232,
34,Hilo,US,69.80,19.7297,-155.0900,
37,Airai,TL,68.79,-8.9266,125.4092,
42,Bredasdorp,ZA,68.00,-34.5322,20.0403,


In [51]:
# 8a. Create the output File (CSV)

output_data_file = "Vacation_Search.csv"

# 8b. Export the City_Data into a csv
clean_data.to_csv(output_data_file, index_label="City_ID")

In [52]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_data[["Lat", "Lng"]]

In [53]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure()
fig.add_layer(marker_layer)

# 11b. Display the figure
fig


Figure(layout=FigureLayout(height='420px'))

In [54]:
# 11a. Add a marker layer for each city to the map. 
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.

max_temp = clean_data["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
market_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))